In [1]:
from dataset import PairedDatasetDouble
pd = PairedDatasetDouble('In/0', 'In/1', 'Out_New', 220)

Length of x_train 0 files: 943
Length of x_train 1 files: 943
Length of y_train files: 943
Item 0 with max: 3482 and min: -2180
Normalized item 0 with max: 1.0 and min: 0.0
Item 1 with max: 1681 and min: -1108
Normalized item 1 with max: 1.0 and min: 0.0
Item 2 with max: 5660 and min: -2212
Normalized item 2 with max: 1.0 and min: 0.0
Item 3 with max: 3564 and min: -1221
Normalized item 3 with max: 1.0 and min: 0.0
Item 4 with max: 3452 and min: -2218
Normalized item 4 with max: 1.0 and min: 0.0
Item 5 with max: 3378 and min: -2213
Normalized item 5 with max: 1.0 and min: 0.0
Item 6 with max: 3744 and min: -2186
Normalized item 6 with max: 1.0 and min: 0.0
Item 7 with max: 3495 and min: -2201
Normalized item 7 with max: 1.0 and min: 0.0
Item 8 with max: 1730 and min: -1078
Normalized item 8 with max: 1.0 and min: 0.0
Item 9 with max: 3420 and min: -1118
Normalized item 9 with max: 1.0 and min: 0.0
Item 10 with max: 3547 and min: -2167
Normalized item 10 with max: 1.0 and min: 0.0
Item 

In [2]:
!conda install -y tensorflow-gpu"=>2.1.0"

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::pandas==1.0.1=py36h0573a6f_0
  - defaults/noarch::jupyterlab==1.2.6=pyhf63ae98_0
  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::python-language-server==0.31.7=py36_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/linux-64::nb_conda==2.2.1=py36_0
  - defaults/noarch::numpydoc==0.9.2=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::nbconvert==5.6.1=py36_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::bokeh==1.4.0=py36_0
  - defaults/noarch::jupyter

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

double_model = load_model('dual_model_final.h5')

with open('model_arch.json', 'r') as f:
    single_model = model_from_json(f.read())
    
single_model.load_weights('model_weights.h5')
print("Single and Double AE Models Loaded")

Single and Double AE Models Loaded


In [18]:
import numpy as np
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
import numpy.ma as ma
import math
import cv2 as cv

def mae(imageA, imageB):

    mae = np.sum(imageB.astype("float") - imageA.astype("float"))
    mae /= float(imageA.shape[0] * imageA.shape[1] * 255) 

    if (mae < 0):
        return mae * -1
    else:
        return mae

def extract_slice_mask(slice):

    ret, thresh = cv.threshold(slice, 0.1, 1, cv.THRESH_BINARY)
    num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(thresh.astype('uint8'))

    max = stats[1, 4]
    largest_index = 1

    for k in np.delete(np.unique(labels_im), 0):

        size = stats[k, 4]
        if (size > max):

            max = size
            largest_index = k

    mask = ma.masked_not_equal(labels_im, largest_index)
    return mask

def extract_lung_mask(volume, size = 128):

    slices = []

    for i in range(0, size):

        masked_slice = extract_slice_mask(volume[i])
        slices.append(masked_slice)

    slices = np.array(slices)
    return slices

def tpfp(truth, prediction):

    h, w, l = truth.shape[0], truth.shape[1], truth.shape[2]

    tp = np.zeros(truth.shape)
    fp = np.zeros(truth.shape)
    fn = np.zeros(truth.shape)
    tn = np.zeros(truth.shape)

    truth_mask = extract_lung_mask(truth)
    pred_mask = extract_lung_mask(prediction)

    for i in range(0, h):

        for j in range(0, w):

            for k in range(0, l):

                if (truth_mask[i, j, k] == 1 and pred_mask[i, j, k] == 1):

                    tp[i, j, k] = 1

                elif (truth_mask[i, j, k] == 1 and pred_mask[i, j, k] == 0):

                    fn[i, j, k] = 1

                elif (truth_mask[i, j, k] == 0 and pred_mask[i, j, k] == 1):

                    fp[i, j, k] = 1

                else:

                    tn[i, j, k] = 1

    tp = ma.masked_values(tp * 100, 0)
    fp = ma.masked_values(fp * 50, 0)
    fn = ma.masked_values(fn, 0)
    
    prdd = pred_mask[64]
    trdd = truth_mask[64]

    #dice = np.sum(prdd[trdd==1])*2.0 / (np.sum(prdd) + np.sum(trdd))
    dice = dice_score(truth_mask, pred_mask)
    return tp, fp, fn, dice


def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    else:
        PIXEL_MAX = 1
        return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def dice_score(im1, im2, empty_score=1.0):

    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

mae_singles = []
mse_singles = []
dice_singles = []
psnr_singles = []
ssim_singles = []

mae_doubles = []
mse_doubles = []
dice_doubles = []
psnr_doubles = []
ssim_doubles = []

for i in range(0, 200):
    
    input_1 = pd.x_train_0[i]
    input_2 = pd.x_train_1[i]
    
    input_1 = np.reshape(input_1, (1, 1024, 1024, 1))
    input_2 = np.reshape(input_2, (1, 1024, 1024, 1))
    
    output_single = single_model.predict(input_1)
    output_double = double_model.predict([input_1, input_2])
    
    output_single = np.reshape(output_single, (128, 128, 128))
    output_double = np.reshape(output_double, (128, 128, 128))
    
    original = pd.y_train[i]
    
    mae_single = mae(original, output_single)
    mae_double = mae(original, output_double)
    
    mse_single = mse(original, output_single)
    mse_double = mse(original, output_double)
    
    ssim_single = ssim(original, output_single)
    ssim_double = ssim(original, output_double)
    
    psnr_single = psnr(original, output_single)
    psnr_double = psnr(original, output_double)
    
    tps, fps, fns, dice_single = tpfp(original, output_single)
    tpd, fpd, fnd, dice_double = tpfp(original, output_double)
    
    mae_doubles.append(mae_double)
    mae_singles.append(mae_single)
    dice_doubles.append(dice_double)
    dice_singles.append(dice_single)
    mse_doubles.append(mse_double)
    mse_singles.append(mse_single)
    psnr_doubles.append(psnr_double)
    psnr_singles.append(psnr_single)
    ssim_doubles.append(ssim_double)
    ssim_singles.append(ssim_single)
    '''
    print("MAE SINGLE " + str(mae_single))
    print("MAE DOUBLE " + str(mae_double))
    print("MSE SINGLE " + str(mse_single))
    print("MSE DOUBLE " + str(mse_double))
    print("PSNR SINGLE " + str(psnr_single))
    print("PSNR DOUBLE " + str(psnr_double))
    print("DICE SINGLE " + str(dice_single))
    print("DICE DOUBLE " + str(dice_double))
    print("SSIM SINGLE " + str(ssim_single))
    print("SSIM DOUBLE " + str(ssim_double))
    '''
    print(str(i))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:154: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:155: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


KeyboardInterrupt: 

In [21]:
from statistics import mean

print("MAE SINGLE " + str(mean(mae_singles)))
print("MAE DOUBLE " + str(mean(mae_doubles)))
print("MSE SINGLE " + str(mean(mse_singles)))
print("MSE DOUBLE " + str(mean(mse_doubles)))
print("PSNR SINGLE " + str(mean(psnr_singles)))
print("PSNR DOUBLE " + str(mean(psnr_doubles)))
print("DICE SINGLE " + str(mean(dice_singles)))
print("DICE DOUBLE " + str(mean(dice_doubles)))
print("SSIM SINGLE " + str(mean(ssim_singles)))
print("SSIM DOUBLE " + str(mean(ssim_doubles)))

MAE SINGLE 0.0012778901083830639
MAE DOUBLE 0.00040961976174969117
MSE SINGLE 0.0011866321434652735
MSE DOUBLE 0.000908615645568086
PSNR SINGLE 29.453823730701973
PSNR DOUBLE 30.624296505262176
DICE SINGLE 0.9607905091564188
DICE DOUBLE 0.9676135570060295
SSIM SINGLE 0.8892748110819574
SSIM DOUBLE 0.9095534750011499
